# 🎯 Configuração de Trigger: File Arrival (Chegada de Arquivo)

Este notebook demonstra como configurar um **Lakeflow Job** com trigger do tipo **File Arrival** para ingestão automática quando novos arquivos chegam no volume.

## 📋 Objetivo

Criar um job que executa automaticamente a ingestão de `claims*.csv` sempre que um novo arquivo é detectado no diretório `/Volumes/{catalog}/00_landing/sql_server/`.

## 🔄 Fluxo do Trigger File Arrival

1. **Monitoramento**: O Lakeflow Jobs monitora o diretório especificado
2. **Detecção**: Quando um novo arquivo `claims*.csv` é adicionado ao volume
3. **Disparo**: O job é automaticamente executado
4. **Processamento**: O script `ingestion_claim.py` processa todos os arquivos `claims*.csv`

## 📝 Pré-requisitos

- ✅ Catálogo e schemas criados
- ✅ Volume `00_landing` configurado
- ✅ Script `ingestion_claim.py` disponível em `source_to_bronze/`


## 🛠️ Passo 1: Criar o Job no Lakeflow Jobs

1. Acesse **Lakeflow Jobs** no menu lateral do Databricks
2. Clique em **Create Job**
3. Configure o nome: `ingestion_claims_file_arrival`
4. Adicione uma **Task** do tipo **Python Script**


## 📂 Passo 2: Configurar a Task

### Configurações da Task:

**Task Name:** `ingest_claims`

**Task Type:** `Python script`

**Source:** `Workspace file`
- Caminho: `/Workspace/Users/{seu_usuario}/trilha_databricks/aula_05/source_to_bronze/ingestion_claim.py`
- Ou use: `File path` e navegue até o arquivo

**Parameters (Task Variables):**
- `catalog` = `smart_claims_dev`
- `schema` = `01_bronze`


## ⚡ Passo 3: Configurar o Trigger File Arrival

### No painel de configuração do Job:

1. Vá para a seção **Triggers**
2. Clique em **Add trigger**
3. Selecione **File arrival**

### Configurações do Trigger:

**Trigger Type:** `File arrival`

**Path to monitor:** 
```
/Volumes/smart_claims_dev/00_landing/sql_server/
```

**File pattern:** 
```
claims*.csv
```

**Wait for completion:** 
- ✅ Marque esta opção se quiser que o job aguarde a conclusão antes de processar novos arquivos
- Útil para evitar execuções concorrentes

**Min time between triggers (seconds):**
- Exemplo: `60` (aguarda 60 segundos entre execuções)
- Evita múltiplas execuções se vários arquivos chegarem ao mesmo tempo


## 🧪 Passo 4: Testar o Trigger

### Simular chegada de arquivo:

Execute o código abaixo para simular a adição de um novo arquivo no volume:


In [ ]:
# Simular chegada de novo arquivo
# Este código cria um arquivo CSV de teste no volume

import os
from datetime import datetime

catalog = "smart_claims_dev"
volume_path = f"/Volumes/{catalog}/00_landing/sql_server/"

# Criar um arquivo de teste com timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
test_file = f"{volume_path}claims_test_{timestamp}.csv"

# Exemplo de dados de teste
test_data = """claim_no,policy_no,claim_date,months_as_customer,injury,property,vehicle,total
TEST001,POL001,2024-01-15,12,0,1000,5000,6000"""

# Escrever arquivo (simula chegada de novo arquivo)
with open(test_file, "w") as f:
    f.write(test_data)

print(f"✅ Arquivo de teste criado: {test_file}")
print(f"📊 O job deve ser disparado automaticamente em alguns segundos!")
print(f"🔍 Verifique o status do job em: Lakeflow Jobs > ingestion_claims_file_arrival")


## 📊 Passo 5: Monitorar Execuções

### Verificar execuções do job:

1. Acesse **Lakeflow Jobs** > `ingestion_claims_file_arrival`
2. Veja o histórico de execuções na aba **Runs**
3. Cada execução mostra:
   - **Trigger:** File arrival
   - **Arquivo detectado:** Nome do arquivo que disparou o job
   - **Status:** Success, Failed, ou Running
   - **Tempo de execução**

### Verificar logs:

- Clique em uma execução para ver logs detalhados
- Verifique se todos os arquivos `claims*.csv` foram processados
- Confirme que a tabela `smart_claims_dev.01_bronze.claims` foi atualizada


## ✅ Vantagens do Trigger File Arrival

- ✅ **Automação completa**: Não precisa executar manualmente
- ✅ **Tempo real**: Processa dados assim que chegam
- ✅ **Eficiência**: Só executa quando há novos arquivos
- ✅ **Confiabilidade**: Evita perda de dados por atraso no processamento

## ⚠️ Considerações Importantes

- O trigger monitora o diretório continuamente
- Múltiplos arquivos podem disparar múltiplas execuções
- Use `Min time between triggers` para evitar sobrecarga
- O padrão glob `claims*.csv` processa TODOS os arquivos correspondentes a cada execução
